In [1]:
import os
import polars as pl

os.chdir("../../../")

# predictables\encoding\src\lagged_mean_encoding\_dynamic_rolling_sum.py
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_sum import (
    DynamicRollingSum,
    dynamic_rolling_sum,
)
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_count import (
    DynamicRollingCount,
)
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_mean import (
    DynamicRollingMean,
)

from predictables.encoding.src.lagged_mean_encoding.credibility import vhm

In [2]:
!pwd

/app


In [3]:
# lf = pl.scan_parquet(
#     "predictables/encoding/tests/ts_testing_df.parquet"
# ).with_row_index()
import polars.selectors as cs

lf = pl.read_excel("ts_testing.xlsx", sheet_name="Values", engine="calamine").lazy()
lf = lf.with_columns(
    [
        pl.col(col).cast(pl.Float64).round(6).name.keep()
        for col in lf.select(cs.float()).columns
    ]
).with_row_index()
# lf.select([pl.col("30_days_prior"), pl.col("60_days_prior")]).select(
#     [
#         pl.col("60_days_prior").name.keep(),
#         pl.date_ranges(pl.col("60_days_prior"), pl.col("30_days_prior"), "1d").alias(
#             "date_list"
#         ),
#     ]
# ).collect().with_columns(
#     [pl.col("date_list").list.eval(pl.element().dt.to_string("%Y-%m-%d"))]
# )
lf.collect().write_parquet("ts_testing.parquet")
lf.collect()

index,date,trans_id,product_cat,product_subcat,product_code,prob,hit,30_days_prior,60_days_prior,90_days_prior,total_30_30_sum,total_30_60_sum,total_60_30_sum,total_30_30_average,total_30_60_average,total_60_30_average,cat_30_30_average,cat_30_60_average,cat_60_30_average,subcat_30_30_average,subcat_30_60_average,subcat_60_30_average,code_30_30_average,code_30_60_average,code_60_30_average,total_30_30_average_laplace(1)_smoothed,total_30_60_average_laplace(1)_smoothed,total_60_30_average_laplace(1)_smoothed,cat_30_30_average_laplace(1)_smoothed,cat_30_60_average_laplace(1)_smoothed,cat_60_30_average_laplace(1)_smoothed,subcat_30_30_average_laplace(1)_smoothed,subcat_30_60_average_laplace(1)_smoothed,subcat_60_30_average_laplace(1)_smoothed,code_30_30_average_laplace(1)_smoothed,code_30_60_average_laplace(1)_smoothed,code_60_30_average_laplace(1)_smoothed,cred_wtd_30_30_average,cred_wtd_30_60_average,cred_wtd_60_30_average,cred_wtd_30_30_Z,cred_wtd_30_60_Z,cred_wtd_60_30_Z,cred_wtd_30_30_n,cred_wtd_30_60_n,cred_wtd_60_30_n
u32,date,i64,str,str,str,f64,i64,date,date,date,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64
0,2020-01-01,1,"""D""","""f""","""D-f""",0.362041,1,2019-12-02,2019-11-02,2019-10-03,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
1,2020-01-02,1,"""B""","""f""","""B-f""",0.669999,1,2019-12-03,2019-11-03,2019-10-04,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
2,2020-01-03,1,"""A""","""d""","""A-d""",0.795662,0,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
3,2020-01-03,2,"""A""","""e""","""A-e""",0.764309,1,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
4,2020-01-04,1,"""D""","""c""","""D-c""",0.41674,1,2019-12-05,2019-11-05,2019-10-06,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2554,2023-08-27,2,"""A""","""c""","""A-c""",0.96099,0,2023-07-28,2023-06-28,2023-05-29,14,27,13,0.23729,0.23684,0.23636,0.07692,0.0,0.0,0.0,0.18182,0.18182,0.0,0.0,0.0,0.25,0.24348,0.25,0.14286,0.07143,0.07143,0.09091,0.25,0.25,0.2,0.25,0.25,0.20238,0.14312,0.18304,0.44444,0.58333,0.375,4,7,3
2555,2023-08-28,1,"""B""","""c""","""B-c""",0.912941,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.11765,0.0625,0.0625,0.0,0.2,0.2,0.0,0.0,0.0,0.25424,0.24561,0.25,0.16667,0.11765,0.11765,0.09091,0.27273,0.27273,0.16667,0.25,0.25,0.21046,0.16687,0.20037,0.5,0.61538,0.375,5,8,3
2556,2023-08-28,2,"""C""","""d""","""C-d""",0.767156,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.26667,0.25,0.25,0.25,0.33333,0.33333,0.5,0.5,0.5,0.25424,0.24561,0.25,0.3125,0.30769,0.30769,0.30769,0.38462,0.38462,0.6,0.6,0.6,0.28013,0.28381,0.27564,0.44444,0.61538,0.44444,4,8,4


In [4]:
from predictables.encoding.src.lagged_mean_encoding.sum.format_columns import (
    _format_date_col,
    _format_value_col,
)
from predictables.encoding.src.lagged_mean_encoding.sum.get_date_list_col import (
    _get_date_list_col,
)

index_col, date_col, x_col, offset, window = "index", "date", "hit", 30, 30

lf_order = lf.select([index_col, date_col])

lf_ = lf.select([pl.col(index_col), pl.col(date_col), pl.col(x_col)])

lf_ = _format_date_col(lf_, date_col)
lf_ = _format_value_col(lf_, x_col)

lf_ = _get_date_list_col(lf_, date_col, offset, window)  # .with_columns([
#     pl.col('date_list').list.min().alias('min_date'),
#     pl.col('date_list').list.max().alias('max_date'),
# ]).with_columns([
#     (pl.col(date_col) - pl.col('min_date')).alias('date - min'),
#     (pl.col(date_col) - pl.col('max_date')).alias('max - date')
# ])
mapping = (
    lf_.select([date_col, x_col])
    .group_by(date_col)
    .agg(pl.col(x_col).sum())
    .sort(date_col)
)

assert (
    mapping.join(
        lf.select(["date", "hit"]).group_by("date").agg(pl.col("hit").sum()),
        how="left",
        on="date",
    )
    .collect()
    .with_columns(
        [(pl.col("hit") - pl.col("hit_right").cast(pl.Float64)).alias("diff")]
    )
    .filter(pl.col("diff") != 0)
    .shape[0]
    == 0
)

# mapping.join(lf_.select(['date', 'date_list', 'hit']).explode('date_list').group_by('date').agg(pl.col('hit').sum().alias('hitsum')), how='left', left_on='date', right_on="date").collect()
# lf_.collect()
# mapping.collect()

lf_ = (
    lf_.explode("date_list")
    .join(mapping, how="left", left_on="date_list", right_on=date_col)
    .fill_null(0)
    .drop(["date_list", "min_date", "max_date"])
    .group_by([index_col, date_col])
    .agg(pl.col(f"{x_col}_right").sum().alias(f"rolling_{x_col}"))
    .sort("index")
)

lf_.join(
    lf.select(["index", "total_30_30_sum"]), how="left", on="index"
).collect().with_columns(
    [(pl.col("rolling_hit") - pl.col("total_30_30_sum").cast(pl.Float64)).alias("diff")]
)  # .filter(pl.col('diff') != 0)

index,date,rolling_hit,total_30_30_sum,diff
u32,date,f64,i64,f64
0,2020-01-01,0.0,0,0.0
1,2020-01-02,0.0,0,0.0
2,2020-01-03,0.0,0,0.0
3,2020-01-03,0.0,0,0.0
4,2020-01-04,0.0,0,0.0
…,…,…,…,…
2554,2023-08-27,14.0,14,0.0
2555,2023-08-28,14.0,14,0.0
2556,2023-08-28,14.0,14,0.0


In [5]:
lf1 = (
    DynamicRollingCount()
    .lf(lf)
    .x_col("hit")
    .date_col("date")
    .index_col("index")
    # .cat_col("product_cat")
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_COUNT")
).run()

In [6]:
lf2 = (
    DynamicRollingSum()
    .lf(lf1)
    .x_col("hit")
    .date_col("date")
    .index_col("index")
    # .cat_col("product_cat")
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_SUM")
).run()

lf2 = lf2.with_columns(
    [
        pl.when(pl.col("ROLLING_COUNT(count[ALL])[lag:30/win:30]") == 0)
        .then(0)
        .otherwise(
            (
                pl.col("ROLLING_SUM(hit[ALL])[lag:30/win:30]")
                / pl.col("ROLLING_COUNT(count[ALL])[lag:30/win:30]")
            ).round(5)
        )
        .alias("rolling_average")
    ]
).drop(
    ["ROLLING_COUNT(count[ALL])[lag:30/win:30]", "ROLLING_SUM(hit[ALL])[lag:30/win:30]"]
)
# .with_columns([
#     (pl.col('total_30_30_average') - pl.col('rolling_average')).alias('diff')
# ]).filter(pl.col('diff') != 0)

lf2.collect()

index,date,trans_id,product_cat,product_subcat,product_code,prob,hit,30_days_prior,60_days_prior,90_days_prior,total_30_30_sum,total_30_60_sum,total_60_30_sum,total_30_30_average,total_30_60_average,total_60_30_average,cat_30_30_average,cat_30_60_average,cat_60_30_average,subcat_30_30_average,subcat_30_60_average,subcat_60_30_average,code_30_30_average,code_30_60_average,code_60_30_average,total_30_30_average_laplace(1)_smoothed,total_30_60_average_laplace(1)_smoothed,total_60_30_average_laplace(1)_smoothed,cat_30_30_average_laplace(1)_smoothed,cat_30_60_average_laplace(1)_smoothed,cat_60_30_average_laplace(1)_smoothed,subcat_30_30_average_laplace(1)_smoothed,subcat_30_60_average_laplace(1)_smoothed,subcat_60_30_average_laplace(1)_smoothed,code_30_30_average_laplace(1)_smoothed,code_30_60_average_laplace(1)_smoothed,code_60_30_average_laplace(1)_smoothed,cred_wtd_30_30_average,cred_wtd_30_60_average,cred_wtd_60_30_average,cred_wtd_30_30_Z,cred_wtd_30_60_Z,cred_wtd_60_30_Z,cred_wtd_30_30_n,cred_wtd_30_60_n,cred_wtd_60_30_n,count,rolling_average
u32,date,i64,str,str,str,f64,i64,date,date,date,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i32,f64
0,2020-01-01,1,"""D""","""f""","""D-f""",0.362041,1,2019-12-02,2019-11-02,2019-10-03,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0,1,0.0
1,2020-01-02,1,"""B""","""f""","""B-f""",0.669999,1,2019-12-03,2019-11-03,2019-10-04,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0,1,0.0
2,2020-01-03,1,"""A""","""d""","""A-d""",0.795662,0,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0,1,0.0
3,2020-01-03,2,"""A""","""e""","""A-e""",0.764309,1,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0,1,0.0
4,2020-01-04,1,"""D""","""c""","""D-c""",0.41674,1,2019-12-05,2019-11-05,2019-10-06,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0,1,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2554,2023-08-27,2,"""A""","""c""","""A-c""",0.96099,0,2023-07-28,2023-06-28,2023-05-29,14,27,13,0.23729,0.23684,0.23636,0.07692,0.0,0.0,0.0,0.18182,0.18182,0.0,0.0,0.0,0.25,0.24348,0.25,0.14286,0.07143,0.07143,0.09091,0.25,0.25,0.2,0.25,0.25,0.20238,0.14312,0.18304,0.44444,0.58333,0.375,4,7,3,1,0.23729
2555,2023-08-28,1,"""B""","""c""","""B-c""",0.912941,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.11765,0.0625,0.0625,0.0,0.2,0.2,0.0,0.0,0.0,0.25424,0.24561,0.25,0.16667,0.11765,0.11765,0.09091,0.27273,0.27273,0.16667,0.25,0.25,0.21046,0.16687,0.20037,0.5,0.61538,0.375,5,8,3,1,0.24138
2556,2023-08-28,2,"""C""","""d""","""C-d""",0.767156,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.26667,0.25,0.25,0.25,0.33333,0.33333,0.5,0.5,0.5,0.25424,0.24561,0.25,0.3125,0.30769,0.30769,0.30769,0.38462,0.38462,0.6,0.6,0.6,0.28013,0.28381,0.27564,0.44444,0.61538,0.44444,4,8,4,1,0.24138


In [7]:
lf2.select(
    [
        pl.col("index"),
        pl.col("date"),
        pl.col("hit"),
        pl.col("total_30_30_average"),
        # pl.col('ROLLING_COUNT(count[ALL])[lag:30/win:30]'),
        # pl.col('ROLLING_SUM(hit[ALL])[lag:30/win:30]'),
        pl.col("rolling_average"),
    ]
).collect()

index,date,hit,total_30_30_average,rolling_average
u32,date,i64,f64,f64
0,2020-01-01,1,0.0,0.0
1,2020-01-02,1,0.0,0.0
2,2020-01-03,0,0.0,0.0
3,2020-01-03,1,0.0,0.0
4,2020-01-04,1,0.0,0.0
…,…,…,…,…
2554,2023-08-27,0,0.23729,0.23729
2555,2023-08-28,0,0.24138,0.24138
2556,2023-08-28,0,0.24138,0.24138


In [8]:
lf3 = (
    DynamicRollingMean()
    .lf(lf2)
    .numerator_col("hit")
    .denominator_col("count")
    .date_col("date")
    .index_col("index")
    .cat_col("product_subcat")
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_MEAN")
).run()

lf3.select(
    [
        pl.col("index"),
        pl.col("date"),
        pl.col("hit"),
        pl.col("subcat_30_30_average"),
        pl.col("ROLLING_MEAN(hit[product_subcat])[lag:30/win:30]"),
    ]
).collect()

index,date,hit,subcat_30_30_average,ROLLING_MEAN(hit[product_subcat])[lag:30/win:30]
u32,date,i64,f64,f64
0,2020-01-01,1,0.0,0.0
1,2020-01-02,1,0.0,0.0
2,2020-01-03,0,0.0,0.0
3,2020-01-03,1,0.0,0.0
4,2020-01-04,1,0.0,0.0
…,…,…,…,…
2554,2023-08-27,0,0.0,0.0
2555,2023-08-28,0,0.0,0.0
2556,2023-08-28,0,0.25,0.25
